### Библиотеки / данные

импортируем библиотеки numpy и pandas

In [ ]:
import numpy as np
import pandas as pd

задаем некоторые настройки pandas, регулирующие формат вывода

In [ ]:
pd.options.display.max_rows = 10

### Слияние датафреймов

**Типы соединений**
<img src='..\images\join-types-merge-names.jpg'/>

**Аргументы функции merge**


| Аргумент        | Описание
| ------------- |:-------------:|
|left|Объект DataFrame в левой части операции слияния|
|right|Объект DataFrame в правой части операции слияния|
|how|Допустимые значения: 'inner', 'outer', 'left', 'right '|
|on|Имена столбцов, по которым производится соединение. Должны присутствовать в обоих объектах DataFrame. Если не заданы и не указаны никакие другие ключи соединения, то используются имена столбцов, общих для обоих объектов|
|left_on|Столбцы левого DataFrame, используемые как ключи соединения|
|right_on|Столбцы правого DataFrame, используемые как ключи соединения|
|left index|Использовать индекс строк левого DataFrame в качестве его ключа соединения (или нескольких ключей в случае мультииндекса)|
|right_index|То же, что left_index, но для правого DataFrame|
|sort|Сортировать слитые данные лексикографически по ключам соединения по умолчанию True. Иногда при работе с большими наборами данных лучше отключить
|suffixes|Кортеж строк, которые дописываются в конец совпадающих имен столбцов;
|сору|Если равен False, то в некоторых особых случаях разрешается не копировать данные в результирующую структуру.|

для примеров:

In [ ]:
df1 = pd.DataFrame({'key': ['a','a', 'a', 'b', 'b', 'c', 'd'],
                    'data1': range(7)},
                    index = range(3,10))
df2 = pd.DataFrame({'key': ['a', 'b', 'b', 'f'],
                    'data2': range(4)})

In [ ]:
df1

In [ ]:
df2

по умолчанию:

In [ ]:
pd.merge(df1, df2)

по каким столбцам соединять?

In [ ]:
pd.merge(df1, df2, on='key')

разные имена столбцов:

In [ ]:
pd.merge(df1, df2, left_on='data1', right_on='data2')

задание суффиксовов:

In [ ]:
pd.merge(df1, df2, left_on='data1', right_on='data2',suffixes=('_left', '_right'))

соединение по нескольким столбцам:

In [ ]:
pd.merge(df1, df2,
         left_on=['data1', 'key'],
         right_on=['data2', 'key'])

указываем тип соединения:

In [ ]:
pd.merge(df1, df2, how='outer')

In [ ]:
df1

In [ ]:
df2

In [ ]:
pd.merge(df1, df2, left_index=True, right_on='data2')

In [ ]:
pd.merge(df1, df2, left_on='data1', right_index=True)

In [ ]:
pd.merge(df1, df2, left_index=True, right_index=True)

### Конкатенация датафреймов

для примеров:

In [ ]:
df1 = pd.DataFrame(index = range(3), data = np.arange(9).reshape(3, 3), 
                   columns=['a', 'b', 'c'])
df2 = pd.DataFrame(index = range(2,5), data = np.arange(9, 18).reshape(3, 3), 
                   columns=['a', 'c', 'd'])

In [ ]:
df1

In [ ]:
df2

По умолчанию:

In [ ]:
pd.concat([df1, df2], sort=True)

In [ ]:
pd.concat([df1, df2], axis=0, sort=True, join='inner')

По оси столбцов:

In [ ]:
pd.concat([df1, df2], axis=1, sort=True)

In [ ]:
df1

In [ ]:
df2

### "Long” и “Wide”

считываем данные акселерометра ("Long")

In [ ]:
long_format = pd.read_csv("data/accel.csv")
long_format

"Wide" формат, используем unstack:

In [ ]:
tmp = long_format.set_index(['interval','axis'])
tmp

In [ ]:
tmp = tmp.unstack()
tmp

"Wide" формат, используем pivot:

In [ ]:
wide_format = long_format.pivot(index='interval', 
                                columns='axis', 
                                values='reading')

wide_format

Из "Wide" в "Long" формат:

In [ ]:
long_format_stack = wide_format.stack()
long_format_stack

создаем две копии данных акселерометра, по одной для каждого пользователя

In [ ]:
user1 = long_format.copy()
user2 = long_format.copy()

добавляем столбец who в каждую копию

In [ ]:
user1['who'] = 'Mike'
user2['who'] = 'David'

отмасштабируем данные user2

In [ ]:
user2['reading'] *= 100

In [ ]:
user1.head()

In [ ]:
user2.head()

организуем данные так, чтобы получить иерархический индекс строк

In [ ]:
multi_user_long_format = pd.concat([user1, user2]).set_index(['who', 'interval', 'axis'])
multi_user_long_format

пример для повторения извлекаем все показания по всем осям и по всем пользователям при interval == 1

In [ ]:
multi_user_long_format.xs(1, level='interval')

выполняем расстыковку, в результате 2-й уровень индекса строк (уровень axis) стал уровнем индекса столбцов

In [ ]:
multi_user_wide_format = multi_user_long_format.unstack()
multi_user_wide_format

параметр level:

In [ ]:
multi_user_long_format.unstack(level = 1)